### Typings

#### Type aliases

In [1]:
Vector = list[float]

def scale(scalar: float, vector: Vector):
    return [scalar * num for num in vector]

In [2]:
new_vector = scale(2.0, [1.0, -4.2, 5.4])
new_vector

[2.0, -8.4, 10.8]

#### New type

The static type checker will treat the new type as if it were a subclass of the original type. This is useful in helping catch logical errors

In [3]:
from typing import NewType

UserId = NewType('UserId', int)
some_id = UserId(524313)

In [4]:
def get_user_name(user_id: UserId) -> str:
    ...

# typechecks
user_a = get_user_name(UserId(42351))

# does not typecheck; an int is not a UserId
user_b = get_user_name(-1)

In [5]:
# 'output' is of type 'int', not 'UserId'
output = UserId(23413) + UserId(54341)
output, type(output)

(77754, int)

In [6]:
user_id = 1
UserId(user_id) is user_id

True

In [7]:
from typing import NewType
from contextlib import suppress

UserId = NewType('UserId', int)

# Fails at runtime and does not typecheck
with suppress(TypeError):
    class AdminUserId(UserId): pass

In [8]:
from typing import NewType

UserId = NewType('UserId', int)

ProUserId = NewType('ProUserId', UserId)

#### Callable

In [9]:
from collections.abc import Callable

In [10]:
def feeder(get_next_item: Callable[[], str]) -> None:
    pass

In [11]:
def async_query(on_success: Callable[[int], None],
                on_error: Callable[[int, Exception], None]) -> None:
    pass

In [12]:
CallableWithoutArguments = Callable[..., None]

#### Generics

In [13]:
from collections.abc import Mapping, Sequence

def notify_by_email(employees: Sequence[str],
                    overrides: Mapping[str, str]) -> None: ...


In [14]:
from collections.abc import Sequence
from typing import TypeVar

T = TypeVar('T')      # Declare type variable

def first(l: Sequence[T]) -> T:   # Generic function
    return l[0]

In [15]:
from typing import TypeVar, Generic
from logging import Logger

T = TypeVar('T')

class LoggedVar(Generic[T]):
    def __init__(self, value: T, name: str, logger: Logger) -> None:
        self.name = name
        self.logger = logger
        self.value = value

    def set(self, new: T) -> None:
        self.log('Set ' + repr(self.value))
        self.value = new

    def get(self) -> T:
        self.log('Get ' + repr(self.value))
        return self.value

    def log(self, message: str) -> None:
        self.logger.info('%s: %s', self.name, message)

The Generic base class defines `__class_getitem__()` so that `LoggedVar[t]` is valid as a type:

In [16]:
from collections.abc import Iterable

def zero_all_vars(vars: Iterable[LoggedVar[int]]) -> None:
    for var in vars:
        var.set(0)

In [17]:
from typing import TypeVar, Generic
...
# A generic type can have any number of type variables, and type variables may be constrained:
T = TypeVar('T')
S = TypeVar('S', int, str)

class StrangePair(Generic[T, S]): ...

In [18]:
from collections.abc import Sized
from typing import TypeVar, Generic

T = TypeVar('T')

# You can use multiple inheritance with Generic:
class LinkedList(Sized, Generic[T]):
    ...

In [19]:
# When inheriting from generic classes, some type variables could be fixed:
from collections.abc import Mapping
from typing import TypeVar

T = TypeVar('T')

class MyDict(Mapping[str, T]):
    ...

In [20]:
from collections.abc import Iterable

class MyIterable(Iterable): ... # Same as Iterable[Any]

In [22]:
# User defined generic type aliases are also supported.

from collections.abc import Iterable
from typing import TypeVar, Union

S = TypeVar('S')
# Response = Union[Iterable[S], int]
Response = Iterable[S] | int

# Return type here is same as Iterable[str] | int
def response(query: str) -> Response[str]:
    ...

T = TypeVar('T', int, float, complex)
Vec = Iterable[tuple[T, T]]

def inproduct(v: Vec[T]) -> T: # Same as Iterable[tuple[T, T]]
    return sum(x*y for x, y in v)


In [24]:
from typing import Generic, ParamSpec, TypeVar

T = TypeVar('T')
P = ParamSpec('P')

class Z(Generic[T, P]): ...

Z[int, [dict, float]]

__main__.Z[int, (<class 'dict'>, <class 'float'>)]

In [25]:
class X(Generic[P]): ...

X[int, str], X[[int, str]]

(__main__.X[(<class 'int'>, <class 'str'>)],
 __main__.X[(<class 'int'>, <class 'str'>)])

#### The Any type

In [26]:
from typing import Any

In [27]:
a: Any = None
a = []          # OK
a = 2           # OK
s: str = ''
s = a           # OK

def foo(item: Any) -> int:
    # Typechecks; 'item' could be any type,
    # and that type might have a 'bar' method
    item.bar()
    ...

In [31]:
def legacy_parser(text):
    ...
    return data

print(legacy_parser.__annotations__)

# A static type checker will treat the above
# as having the same signature as:
def legacy_parser(text: Any) -> Any:
    ...
    return data

print(legacy_parser.__annotations__)

{}
{'text': typing.Any, 'return': typing.Any}


Contrast the behavior of `Any` with the behavior of `object`. Similar to `Any`, every type is a subtype of `object`. However, unlike `Any`, the reverse is not true: `object `is not a subtype of every other type.

That means when the type of a value is `object`, a type checker will reject almost all operations on it, and assigning it to a variable (or using it as a return value) of a more specialized type is a type error. 

In [35]:
def hash_a(item: object) -> int:
    # Fails; an object does not have a 'magic' method.
    item.magic()
    ...

def hash_b(item: Any) -> int:
    # Typechecks
    item.magic()
    ...
    
# Typechecks, since ints and strs are subclasses of object
hash_a(42)
hash_a("foo")

# Typechecks, since Any is compatible with all types
hash_b(42)
hash_b("foo")

AttributeError: 'int' object has no attribute 'magic'

#### Nominal vs structural subtyping

Initially PEP 484 defined Python static type system as using *nominal subtyping*. This means that a class A is allowed where a class B is expected if and only if A is a subclass of B.

This requirement previously also applied to abstract base classes, such as `Iterable`. The problem with this approach is that a class had to be explicitly marked to support them, which is unpythonic and unlike what one would normally do in idiomatic dynamically typed Python code. 

In [42]:
from collections.abc import Sized, Iterable, Iterator

class Bucket(Sized, Iterable[int]):
    ...
    def __len__(self) -> int: ...
    def __iter__(self) -> Iterator[int]: ...

`PEP 544` allows to solve this problem by allowing users to write the above code without explicit base classes in the class definition, allowing Bucket to be implicitly considered a subtype of both `Sized` and `Iterable[int]` by static type checkers. This is known as `structural subtyping`(or `static duck-typing`:

In [43]:
from collections.abc import Iterator, Iterable

class Bucket:  # Note: no base classes
    ...
    def __len__(self) -> int: ...
    def __iter__(self) -> Iterator[int]: ...

def collect(items: Iterable[int]) -> int: ...
result = collect(Bucket())  # Passes type check

#### Special typing primitives

##### Any

Every type is compatible with `Any`.
`Any` is compatible with every type.

In [47]:
x: Any = 34 
x = 'd'
def f(x: Any) -> Any:
    return 'x.do()'

f(42)  # typechecks
f('d')  # typechecks
f(None)  # typechecks

'x.do()'

##### NoReturn

Special type indicating that a function never returns. 

In [48]:
from typing import NoReturn

def fail() -> NoReturn:
    raise RuntimeError('no way')
    
with suppress(RuntimeError):
    fail()

##### TypeAlias

In [49]:
from typing import TypeAlias

Factors: TypeAlias = list[int]

#### Special forms

##### Union

 Union type; `Union[X, Y]` is equivalent to `X | Y` and means either X or Y.

In [52]:
from typing import Union

- The arguments must be types and there must be at least one.

- Unions of unions are flattened, e.g.

In [53]:
assert Union[int, Union[float, str]] == Union[int, float, str]

- Unions of a single argument vanish, e.g.

In [54]:
assert Union[str] == str

- Redundant arguments are skipped, e.g.

In [57]:
assert Union[int, str, int] == Union[int, str] == int | str == int | str | int

- When comparing unions, the argument order is ignored, e.g.:

In [59]:
assert Union[int, str] == Union[str, int] == int | str == str | int

- You cannot subclass or instantiate a Union.

In [60]:
class X(Union): ...

TypeError: Cannot subclass typing.Union

In [61]:
Union()

TypeError: Cannot instantiate typing.Union

##### Optional

`Optional[X]` is equivalent to `X | None` (or `Union[X, None]`).

In [63]:
from typing import Optional

In [64]:
def foo(arg: int = 0) -> None:
    ...

def foo(arg: Optional[int] = None) -> None:
    ...

##### Callable

In [69]:
from collections.abc import Callable

In [70]:
def foo(x: int) -> str:
    ...

func: Callable[[int], str] = foo

In [71]:
from collections.abc import Mapping

def foo(*args: Any, **kwargs: Mapping) -> int:
    ...

func: Callable[..., int] = foo

Callables which take other callables as arguments may indicate that their parameter types are dependent on each other using `ParamSpec`.

Additionally, if that callable adds or removes arguments from other callables, the `Concatenate` operator may be used.

##### Concatenate

Used with `Callable` and `ParamSpec` to type annotate a higher order callable which adds, removes, or transforms parameters of another callable. Usage is in the form `Concatenate[Arg1Type, Arg2Type, ..., ParamSpecVariable]`. Concatenate is currently only valid when used as the first argument to a `Callable`. The last parameter to `Concatenate` must be a `ParamSpec`.

In [75]:
from collections.abc import Callable
from threading import Lock
from typing import Concatenate, ParamSpec, TypeVar

P = ParamSpec('P')
R = TypeVar('R')

# Use this lock to ensure that only one thread is executing a function
# at any time.
my_lock = Lock()

def with_lock(f: Callable[Concatenate[Lock, P], R]) -> Callable[P, R]:
    '''A type-safe decorator which provides a lock.'''
    global my_lock
    def inner(*args: P.args, **kwargs: P.kwargs) -> R:
        # Provide the lock as the first argument.
        return f(my_lock, *args, **kwargs)
    return inner

@with_lock
def sum_threadsafe(lock: Lock, numbers: list[float]) -> float:
    '''Add a list of numbers together in a thread-safe manner.'''
    with lock:
        return sum(numbers)

# We don't need to pass in the lock ourselves thanks to the decorator.
sum_threadsafe([1.1, 2.2, 3.3])

6.6

##### Type

In [76]:
from typing import Type

In [77]:
x = 3   # int
x = int   # Type[int]
x = type(3)  # Type[int]

Note that `Type[C]` is covariant:

In [79]:
class User: ...
class BasicUser(User): ...
class AdvancedUser(User): ...
    
# Accepts User, BasicUser, AdvancedUser, ...
def make_new_user(user_class: Type[User]) -> User:
    # ...
    return user_class()

The only legal parameters for `Type` are `classes`, `Any`, `type variables`, and `unions` of any of these types. For example:

In [81]:
class TeamUser(User): ...

def make_non_team_user(user_class: Type[Union[BasicUser, AdvancedUser]]) -> User: ...

In [86]:
Type, Type[Any], type

(typing.Type, typing.Type[typing.Any], type)

In [87]:
type[BasicUser]

type[__main__.BasicUser]

In [88]:
def make_non_team_user(user_class: type[BasicUser | AdvancedUser]) -> User: ...

##### Literal

A type that can be used to indicate to type checkers that the corresponding variable or function parameter has a value equivalent to the provided literal (or one of several literals). For example:

In [93]:
from typing import Literal

In [94]:
def validate_simple(data: Any) -> Literal[True]:...  # always returns True

In [97]:
MODE = Literal['r', 'rb', 'w', 'wb']
def open_helper(file: str, mode: MODE) -> str:...
    
open_helper('/some/path', 'r')  # Passes type check
open_helper('/other/path', 'typo')  # Error in type checker

##### ClassVar

Special type construct to mark class variables.

As introduced in `PEP 526`, a variable annotation wrapped in `ClassVar` indicates that a given attribute is intended to be used as a class variable and should not be set on instances of that class. 

In [99]:
from typing import ClassVar

In [103]:
class Starship:
    stats: ClassVar[dict[str, int]] = {} # class variable
    damage: int = 10                     # instance variable
    
    def __init__(self, damage: int):
        self.damage = damage

In [104]:
enterprise_d = Starship(3000)
enterprise_d.stats = {} # Error, setting class variable on instance
Starship.stats = {}     # This is OK

##### Final

A special typing construct to indicate to type checkers that a name cannot be re-assigned or overridden in a subclass.

In [107]:
from typing import Final

In [108]:
MAX_SIZE: Final = 9000
MAX_SIZE += 1  # Error reported by type checker

In [109]:
class Connection:
    TIMEOUT: Final[int] = 10

class FastConnector(Connection):
    TIMEOUT = 1  # Error reported by type checker

##### Annotated

In [124]:
from typing import Annotated

[stackoverflow discussion on Annotated](https://stackoverflow.com/questions/68454202/how-to-use-maxlen-of-typing-annotation-of-python-3-9/68489244#68489244)

In [114]:
from dataclasses import dataclass

@dataclass
class ValueRange:
    min: int
    max: int

T1 = Annotated[int, ValueRange(-10, 5)]
T2 = Annotated[T1, ValueRange(-20, 3)]

Passing `include_extras=True` to `get_type_hints()` lets one access the extra annotations at runtime.

In [117]:
from typing import get_type_hints

def foo(x: T1) -> None:...
    
get_type_hints(foo), get_type_hints(foo, include_extras=True)

({'x': int, 'return': NoneType},
 {'x': typing.Annotated[int, ValueRange(min=-10, max=5)], 'return': NoneType})

- The first argument to Annotated must be a valid type

- Multiple type annotations are supported (`Annotated` supports variadic arguments):

In [121]:
@dataclass
class Odd:...

Annotated[int, ValueRange(3, 10), Odd()]

typing.Annotated[int, ValueRange(min=3, max=10), Odd()]

- `Annotated` must be called with at least two arguments ( `Annotated[int]` is not valid)

- The order of the annotations is preserved and matters for equality checks:

In [123]:
assert Annotated[int, ValueRange(3, 10), Odd()] != Annotated[
    int, Odd(), ValueRange(3, 10)
]

- Nested `Annotated` types are flattened, with metadata ordered starting with the innermost annotation:

In [125]:
assert Annotated[Annotated[int, Odd()], ValueRange(2, 3)] == Annotated[int, Odd(), ValueRange(2, 3)]

In [126]:
Annotated[Annotated[int, Odd()], ValueRange(2, 3)]

typing.Annotated[int, Odd(), ValueRange(min=2, max=3)]

- Duplicated annotations are not removed:

In [128]:
assert Annotated[int, Odd(), Odd()] != Annotated[int, Odd()]

- Annotated can be used with nested and generic aliases:

In [130]:
@dataclass
class MaxLen:
    len: int
        
T = TypeVar('T')
Vec = Annotated[list[tuple[T, T]], MaxLen(10)]
V = Vec[int]

V == Annotated[list[tuple[int, int]], MaxLen(10)]

True

##### TypeGuard